# 🦜🔗 LCEL introduction

## What is LCEL?

<h4>
- LangChain Expression Language, or LCEL, is a declarative way to compose chains easily. <br>
- This is a new sintaxe for creating chains and agents.
</h4>

<h3>Pipe Syntax</h3>

Composition (chains) can now use the Linux pipe syntax.

Example:
```python
chain = prompt | llm | OutputParser
```

<h3>Interface</h3>

- Components implement the <u>Runnable</u> protocol.
- Common methods include invoke, stream, and batch.

<h3>Common I/O</h3>

|Component     |Input Type                                  | Output Type         |
|---           |---                                         |---                  |
|Prompt        |Dictionary                                  |Prompt Value         |
|Retriever     |Single String                               |List of Documents    |
|LLM           |String, list of messages, or a prompt value |String               |
|ChatModel     |String, list of messages, or a prompt value |ChatMessage          |
|Tool          |String or Dictionary                        |Tool dependent       |
|OutputParser  |Output of LLM or ChatModel                  |Parser dependent     |

<h3>Runnable Support</h3>

- Async, Batch, and streaming support out of the box.
- Fallbacks.
- Parallelism.
  - LLM calls can be time-consuming
  - Any components that can be run in parallel are.
- Logging is built in.


## Setup

In [2]:
import os
from openai import OpenAI

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.vectorstores import FAISS

from util import local_settings
from IPython.display import display, HTML

display(HTML(f"""
<h1>LCEL Introduction</h1>
✅ Libraries loaded successfully <br>
✅ OpenAI Key assigned (<font color="LightGreen">...{local_settings.OPENAI_API_KEY[10:-15]}...</font>)
"""))

## 🧑‍🍳 Example 1: Simple chain

### Prompt, Model, and Output Parser

In [3]:
prompt = ChatPromptTemplate.from_template(
    """tell me a short joke about {topic}. After generate a joke, please also translate to portuguese"""
)
model = ChatOpenAI()
output_parser = StrOutputParser()

### Pipe syntax

In [4]:
chain = prompt | model | output_parser

### Invoke the chain

In [5]:
result = chain.invoke({"topic" : "Chicken"})

print(result)

Why did the chicken go to the seance? To talk to the other side! 

Por que a galinha foi à sessão espírita? Para falar com o outro lado!


## 🧑‍🍳 Example 2: 

In [6]:
from typing import List
from langchain.schema import BaseOutputParser


class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

In [7]:

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

result = chain.invoke({"text": "colors"})

result

['red', 'blue', 'green', 'yellow', 'purple']

## Example 3

In [8]:
embeddings = OpenAIEmbeddings()

texts =  [
    "Harrison worked at kensho",
    "bears like to eat honey",
    "Each day is a new opportunity for growth and success.",
    "I believe in my abilities and trust the journey I am on.",
    "I radiate positivity and attract abundance into my life.",
    "I am deserving of love, happiness, and all the good things life has to offer.",
]

retriever = FAISS.from_texts(texts, embeddings)

retriever

In [9]:

retriever.similarity_search("Where did Harrison work?")

[Document(page_content='Harrison worked at kensho'),
 Document(page_content='I believe in my abilities and trust the journey I am on.'),
 Document(page_content='bears like to eat honey'),
 Document(page_content='Each day is a new opportunity for growth and success.')]

In [10]:
prompt = f"Where did Harrison work?"

docs =  retriever.similarity_search(prompt)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

